In [1]:
# First load some libraries, including History Matching
%load_ext autoreload
%autoreload 2
import re, os
from history_matching import HistoryMatching, HistoryMatchingCut, quick_read, Basis
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pycuda

sns.set(font_scale=3)
sns.set_style("white")

C:\conda\envs\history_matching\lib\site-packages\skcuda\cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


## Select samples for the next iteration
Now that the emulators are fit, it's time to select samples for the next iteration.
Here we use a class called HistoryMatchingCut.  It's job is to conduct rejection sampling in order to find n_samples_per_iteration new samples that are not implausible with respect to the emulation on this wave or any previous wave.  Resulting samples are saved to disk, so we'll load them in just to make a plot.

In [2]:
iteration = int(re.search(r'iter(\d+)', os.getcwd()).group(1)) # Index of the current iteration
n_samples_to_generate_for_next_iter = 200 # Number of simulations to conduct on this iteration
params_file = os.path.join('..', 'Params.xlsx')
param_info = quick_read(params_file, 'Params').set_index('Name')

Reading Params from ..\Params.xlsx


In [ ]:
### Cut #######################################################################
print("="*80, "\nCut\n", "="*80)
###############################################################################
# History Matching!
hmc = HistoryMatchingCut(
    cut_dir = 'Cuts',
    iteration = int(re.search(r'iter(\d+)', os.getcwd()).group(1))
)

(_, rejected_percent) = hmc.cut(num_desired_candidates=n_samples_to_generate_for_next_iter, constraint = None)

Cut
Reading iteration 2 . cut 500_8_W2_Sugungum
Welcome to IDM History Matching!
Found 200 unique parameter configurations, each of which is repeated 1 time(s).
--> Training with 150 unique parameter configurations ( 150  simulations including replicates)
--> Testing  with 50  unique parameter configurations ( 50 simulations including replicates)
	 Desired Result: 0.4876543209876543
	 Desired Result Var: 0
	 Discrepancy Var: 0
	 Imp Thresh: 3
Using Poisson family
                 Generalized Linear Model Regression Results                  
Dep. Variable:             Sim_Result   No. Observations:                  150
Model:                            GLM   Df Residuals:                      132
Model Family:                 Poisson   Df Model:                           17
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -35.232
Date:                Sat, 07 Mar 2020   Deviance:         

Reading iteration 0 . cut 50_8_W2_Sugungum
Welcome to IDM History Matching!
Found 999 unique parameter configurations, each of which is repeated 1 time(s).
--> Training with 749 unique parameter configurations ( 749  simulations including replicates)
--> Testing  with 250  unique parameter configurations ( 250 simulations including replicates)
	 Desired Result: 0.3209876543209876
	 Desired Result Var: 0
	 Discrepancy Var: 0
	 Imp Thresh: 3
Using Poisson family
                 Generalized Linear Model Regression Results                  
Dep. Variable:             Sim_Result   No. Observations:                  749
Model:                            GLM   Df Residuals:                      739
Model Family:                 Poisson   Df Model:                            9
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -356.28
Date:                Sat, 07 Mar 2020   Deviance:            

c:\git\history_matching\history_matching\HistoryMatchingCut.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  plausible_candidates['Yglm'] = self.glm_all[cut].evaluate(plausible_candidates)
c:\git\history_matching\history_matching\HistoryMatchingCut.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  plausible_candidates['Mean_Estimate'] = plausible_candidates['Yglm'] + ret['Mean']
c:\git\history_matching\history_matching\HistoryMatchingCut.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

(0, 9)
Returning early because none of the candidates are plausible.
Test plausibility: 3.8540115356445312
Merge plausibility (needed?): 0.0
Plausible candidates: New = 0 , Tot = 0
--------------------------------------------------------------------------------
LHS Sampling ( 5000 ): 0.01595449447631836
LHS Scaling: 0.0029921531677246094
DataFrame: 0.0010037422180175781
Constraint: 0.0
(5000, 9)
Performing cut: iteration 2 , cut 500_8_W2_Sugungum


c:\git\history_matching\history_matching\HistoryMatchingCut.py:91: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(plausible_candidates['Var_Predictive'] + self.hm_params[cut]['desired_result_var'] + self.hm_params[cut]['discrepancy_var'] )


(4450, 9)
Performing cut: iteration 1 , cut 50_28_W2_Sugungum
(1, 9)
Performing cut: iteration 0 , cut 50_8_W2_Sugungum
Test plausibility: 420.1066393852234
Merge plausibility (needed?): 0.001995563507080078
Plausible candidates: New = 1 , Tot = 1
--------------------------------------------------------------------------------
LHS Sampling ( 5000 ): 0.025917768478393555
LHS Scaling: 0.006982326507568359
DataFrame: 0.001998424530029297
Constraint: 0.0009922981262207031
(5000, 9)
Performing cut: iteration 2 , cut 500_8_W2_Sugungum
(4398, 9)
Performing cut: iteration 1 , cut 50_28_W2_Sugungum


In [ ]:
candidates = pd.read_csv('Candidates_for_iter%d.csv'%(iteration+1))

g = sns.PairGrid(candidates)
g.map_upper(plt.scatter, marker='+', linewidths=0.1, s=100)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False);

names = param_info.index.values.tolist()
for col in range(len(names)):
    for row in range(len(names)):
        if row == col:
            continue
        xvar = names[col]
        yvar = names[row]
        #print('B', col, xvar, yvar, g.axes[row, col].get_xlim(), g.axes[row, col].get_ylim())
        g.axes[row,col].set_xlim( param_info.loc[xvar, ['Min','Max']].values.tolist() )
        g.axes[row,col].set_ylim( param_info.loc[yvar, ['Min','Max']].values.tolist() )